In [68]:
import openai
import config
import os 
import json
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.document_loaders import JSONLoader, DirectoryLoader, UnstructuredHTMLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

In [223]:
from langchain.document_loaders import BSHTMLLoader
loader = BSHTMLLoader("/scratch/ac.pgupta/convLLM/kbasedocs/getting-started_browsers.html")
data = loader.load()
data

[Document(page_content='\n\n\n\n\nSupported Browsers - KBase Documentation\n\n\n\n\n\n\n\n\n\n\nKBase DocumentationSearch⌃KKBase DocumentationKBase Terms & ConditionsGetting StartedSigning Up and Signing InSupported BrowsersNarrative Quick StartNarrative Interface User GuideFAQsWorking with DataUsing AppsRunning Common WorkflowsTroubleshootingDeveloping AppsExternal LinksKBase Narrative InterfaceKBase web siteKBase App Catalogkbase.usPowered By GitBookSupported BrowsersThe KBase website and Narrative Interface should work on most modern web browsers, but we primarily focus on supporting Chrome.Supported BrowsersThe following desktop web browsers have been tested against the KBase web site and Narrative user interface. If you experience any issue using these browsers with our site please report it through the Help Board. OSSupported BrowsersMacChromeFirefoxSafari 10 (macOS Sierra) and upWindowsFirefoxMicrosoft Edge - not recommendedLinuxChromeFirefoxAd blockers may prevent KBase from wo

In [225]:
print(data[0].page_content)






Supported Browsers - KBase Documentation










KBase DocumentationSearch⌃KKBase DocumentationKBase Terms & ConditionsGetting StartedSigning Up and Signing InSupported BrowsersNarrative Quick StartNarrative Interface User GuideFAQsWorking with DataUsing AppsRunning Common WorkflowsTroubleshootingDeveloping AppsExternal LinksKBase Narrative InterfaceKBase web siteKBase App Catalogkbase.usPowered By GitBookSupported BrowsersThe KBase website and Narrative Interface should work on most modern web browsers, but we primarily focus on supporting Chrome.Supported BrowsersThe following desktop web browsers have been tested against the KBase web site and Narrative user interface. If you experience any issue using these browsers with our site please report it through the Help Board. OSSupported BrowsersMacChromeFirefoxSafari 10 (macOS Sierra) and upWindowsFirefoxMicrosoft Edge - not recommendedLinuxChromeFirefoxAd blockers may prevent KBase from working in your web browserSome ad-blocker

In [3]:
key = "sk-jaadX..................Y5cAR58M"

In [4]:
os.environ["OPENAI_API_KEY"] = key

In [45]:
html_dir_loader = DirectoryLoader('/scratch/ac.pgupta/convLLM/kbasedocs', glob="**/[!.]*.html", loader_cls=UnstructuredHTMLLoader)
data_html = html_dir_loader.load()
len(data_html)

92

In [222]:
data_html

[Document(page_content='KBase Documentation\n\nHere you will find guidance for getting started with KBase, full documentation for users and App developers, and help with troubleshooting.\n\nHow to cite KBase:\n\nArkin AP, Cottingham RW, Henry CS, Harris NL, Stevens RL, Maslov S, et al. KBase: The United States Department of Energy Systems Biology Knowledgebase. Nature Biotechnology. 2018;36: 566. doi: 10.1038/nbt.4163\n\nFunding:\n\nThis work is supported as part of the Genomic Sciences Program DOE Systems Biology Knowledgebase (KBase) funded by the\n\nU.S. Department of Energy, Office of Science, Office of Biological and Environmental Research\n\nunder Award Numbers DE-AC02-05CH11231, DE-AC02-06CH11357, DE-AC05-00OR22725, and DE-AC02-98CH10886.\n\nNextKBase Terms & Conditions\n\nLast modified \n\n2yr ago', metadata={'source': '/scratch/ac.pgupta/convLLM/kbasedocs/index.html'}),
 Document(page_content='Getting Started\n\nAn introduction to KBase and guide on how to get started.\n\nWhat

In [6]:
llm = ChatOpenAI(temperature=0, model="gpt-4")

In [66]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 20,
    length_function = len,
)
documents_html = text_splitter.split_documents(data_html)
len(documents_html)

408

In [212]:
documents_html[0]

Document(page_content='KBase Documentation\n\nHere you will find guidance for getting started with KBase, full documentation for users and App developers, and help with troubleshooting.\n\nHow to cite KBase:\n\nArkin AP, Cottingham RW, Henry CS, Harris NL, Stevens RL, Maslov S, et al. KBase: The United States Department of Energy Systems Biology Knowledgebase. Nature Biotechnology. 2018;36: 566. doi: 10.1038/nbt.4163\n\nFunding:\n\nThis work is supported as part of the Genomic Sciences Program DOE Systems Biology Knowledgebase (KBase) funded by the\n\nU.S. Department of Energy, Office of Science, Office of Biological and Environmental Research\n\nunder Award Numbers DE-AC02-05CH11231, DE-AC02-06CH11357, DE-AC05-00OR22725, and DE-AC02-98CH10886.\n\nNextKBase Terms & Conditions\n\nLast modified \n\n2yr ago', metadata={'source': '/scratch/ac.pgupta/convLLM/kbasedocs/index.html'})

In [221]:
# UnstructuredHTMLLoader DirectoryLoader will inline tables this way
# original page: https://docs.kbase.us/getting-started/browsers
print(documents_html[17].page_content)

Supported Browsers

The KBase website and Narrative Interface should work on most modern web browsers, but we primarily focus on supporting Chrome.

Supported Browsers

The following desktop web browsers have been tested against the KBase web site and Narrative user interface. If you experience any issue using these browsers with our site please report it through the

Help Board

OS Supported Browsers Mac Chrome Firefox Safari 10 (macOS Sierra) and up Windows Firefox Microsoft Edge - not recommended Linux Chrome Firefox

Ad blockers may prevent KBase from working in your web browser

Some ad-blockers and browser extensions may cause errors within the Narrative Interface. We are working to identify issues with commonly used browser extensions, but you may want to disable browser extensions if you encounter issues within KBase.

Mobile


### Few shot example prompt

In [226]:
few_shot_examples = [
  {
    "Instruction": """I have some KBase related docs retrieved from KBase Webpage by langchain html loader. The Department of Energy Systems Biology Knowledgebase (KBase) is a software and data science platform designed to meet the grand challenge of systems biology: predicting and designing biological function.
                I split the docs into small text pieces, based on each text provided, generate 3-5 questions and answers pairs from the text. Be noted, the texts are converted from html file, so some content like the tables are messed up in one line. please try the best to figure this out.
                The questions can be, if a statement is correct or not? what is xxx? how to do xxx in KBase?
                text is:
                KBase Documentation\n\nHere you will find guidance for getting started with KBase, full documentation for users and App developers, and help with troubleshooting.\n\nHow to cite KBase:\n\nArkin AP, Cottingham RW, Henry CS, Harris NL, Stevens RL, Maslov S, et al. KBase: The United States Department of Energy Systems Biology Knowledgebase. Nature Biotechnology. 2018;36: 566. doi: 10.1038/nbt.4163\n\nFunding:\n\nThis work is supported as part of the Genomic Sciences Program DOE Systems Biology Knowledgebase (KBase) funded by the\n\nU.S. Department of Energy, Office of Science, Office of Biological and Environmental Research\n\nunder Award Numbers DE-AC02-05CH11231, DE-AC02-06CH11357, DE-AC05-00OR22725, and DE-AC02-98CH10886.\n\nNextKBase Terms & Conditions\n\nLast modified \n\n2yr ago
                """,

    "Generated content":"""
    ### question: How to cite KBase? ## answer: Arkin AP, Cottingham RW, Henry CS, Harris NL, Stevens RL, Maslov S, et al.
    ### question: What is KBase? ## answer: The United States Department of Energy Systems Biology Knowledgebase.
    ### question: Who supports and funds KBase? ## answer: The United States Department of Energy (DOE)
    """
  },
  {
    "Instruction": """I have some KBase related docs retrieved from KBase Webpage by langchain html loader. The Department of Energy Systems Biology Knowledgebase (KBase) is a software and data science platform designed to meet the grand challenge of systems biology: predicting and designing biological function.
                I split the docs into small text pieces, based on each text provided, generate 3-5 questions and answers pairs from the text. Be noted, the texts are converted from html file, so some content like the tables are messed up in one line. please try the best to figure this out.
                The questions can be, if a statement is correct or not? what is xxx? how to do xxx in KBase?
                text is:
                Supported Browsers\n\nThe KBase website and Narrative Interface should work on most modern web browsers, but we primarily focus on supporting Chrome.\n\nSupported Browsers\n\nThe following desktop web browsers have been tested against the KBase web site and Narrative user interface. If you experience any issue using these browsers with our site please report it through the\n\nHelp Board\n\nOS Supported Browsers Mac Chrome Firefox Safari 10 (macOS Sierra) and up Windows Firefox Microsoft Edge - not recommended Linux Chrome Firefox\n\nAd blockers may prevent KBase from working in your web browser\n\nSome ad-blockers and browser extensions may cause errors within the Narrative Interface. We are working to identify issues with commonly used browser extensions, but you may want to disable browser extensions if you encounter issues within KBase.\n\nMobile\n\nKBase is currently not supported on mobile browsers. The KBase website and services should work with tablet browsers on iPad and Android devices in landscape orientation, but we do not currently test on these platforms so there may be compatibility issues. Watch for future updates as KBase goes responsive and mobile.\n\nInstallation of Supported Browsers\n\nChrome\n\nChrome is the best browser to use for KBase. The Chrome browser is produced by Google, Inc., and is freely available. Chrome operates on a rolling release, meaning that it is updated frequently and it is not easy to track specific versions. Our best advice is to make sure you are using the most recent version available. On Mac and Windows this is built into the Chrome application itself.\n\n\u200bDownload Chrome\u200b\n\nFirefox\n\nFirefox, produced by the non-profit Mozilla Corporation, is a freely available, open source browser for Mac, Windows, Linux, and other systems. Like Chrome, Firefox is updated frequently, so it is not feasible for us to recommend a specific (or even minimum) version. Our best advice is to make sure you are using the most recent version available. Recent versions of Firefox will automatically update themselves on Mac and Windows.\n\n\u200bDownload Firefox\u200b\n\nSafari\n\nSafari is distributed with Mac OS X. (We do not support the Windows version of Safari.) Specific versions of Mac OS X support specific versions of Safari – see this\n\nWikipedia entry\n\nfor details.\n\nMicrosoft Edge\n\nMicrosoft Edge\n\nis the new web browser for Windows 10; it replaces Internet Explorer. You should be able to use KBase on Edge, but this has not been extensively tested.\n\nUnsupported Browsers\n\nKBase is not supported on Internet Explorer. It should work on Microsoft Edge, but it has not been extensively tested. For the best chance of success on Edge, be sure you are using the most recent version.\n\nOlder versions of supported browsers are also not supported. Please ensure that your browser and system are up-to-date before reporting a problem with a supported browser.\n\nOther modern browsers, such as Opera, may work with KBase, but as we do not test with them we cannot make any assurances about their operation.\n\nPreviousAuthentication Update\n\nNextNarrative Quick Start\n\nLast modified \n\n1yr ago
                """,

    "Generated content":
    """
    ### question: What browswers are supported by KBase? ## answer: There are four browswers that are supported by KBase, they are
    1. Chrome:Chrome is the best browser to use for KBase. 
    2. Firefox: Firefox, produced by the non-profit Mozilla Corporation, is a freely available, open source browser for Mac, Windows, Linux, and other systems. 
    3. Safari: Safari is distributed with Mac OS X. (We do not support the Windows version of Safari.)
    4. Microsoft Edge: Microsoft Edgeis the new web browser for Windows 10; it replaces Internet Explorer. You should be able to use KBase on Edge, but this has not been extensively tested.
    ### question: What browswers are supported for Mac? ## answer: Chrome, Firefox and Safari 10 (macOS Sierra) and up
    ### question: Why Kbase is not working in my browswer? ## answer: Ad blockers may prevent KBase from working in your web browserSome ad-blockers and browser extensions may cause errors within the Narrative Interface. We are working to identify issues with commonly used browser extensions, but you may want to disable browser extensions if you encounter issues within KBase.
    """
  }
]

In [227]:
example_prompt = PromptTemplate(input_variables=["Instruction", "Generated content"], template="Instruction: {Instruction}\nGenerated content: {Generated content}")

print(example_prompt.format(**few_shot_examples[0]))

Instruction: I have some KBase related docs retrieved from KBase Webpage by langchain html loader. The Department of Energy Systems Biology Knowledgebase (KBase) is a software and data science platform designed to meet the grand challenge of systems biology: predicting and designing biological function.
                I split the docs into small text pieces, based on each text provided, generate 3-5 questions and answers pairs from the text. Be noted, the texts are converted from html file, so some content like the tables are messed up in one line. please try the best to figure this out.
                The questions can be, if a statement is correct or not? what is xxx? how to do xxx in KBase?
                text is:
                KBase Documentation

Here you will find guidance for getting started with KBase, full documentation for users and App developers, and help with troubleshooting.

How to cite KBase:

Arkin AP, Cottingham RW, Henry CS, Harris NL, Stevens RL, Maslov S, et a

In [228]:
instruction_prompt="""I have some KBase related docs retrieved from KBase Webpage by langchain html loader. The Department of Energy Systems Biology Knowledgebase (KBase) is a software and data science platform designed to meet the grand challenge of systems biology: predicting and designing biological function.
                I split the docs into small text pieces, based on each text provided, generate 3-5 questions and answers pairs from the text. Be noted, the texts are converted from html file, so some content like the tables are messed up in one line. please try the best to figure this out.
                The questions can be, if a statement is correct or not? what is xxx? how to do xxx in KBase?
                text is:"""
prompt = FewShotPromptTemplate(
    examples=few_shot_examples,
    example_prompt=example_prompt,
    suffix="Instruction:  {input}",
    input_variables=["input"]
)
prompt
print(prompt.format(input=instruction_prompt))

Instruction: I have some KBase related docs retrieved from KBase Webpage by langchain html loader. The Department of Energy Systems Biology Knowledgebase (KBase) is a software and data science platform designed to meet the grand challenge of systems biology: predicting and designing biological function.
                I split the docs into small text pieces, based on each text provided, generate 3-5 questions and answers pairs from the text. Be noted, the texts are converted from html file, so some content like the tables are messed up in one line. please try the best to figure this out.
                The questions can be, if a statement is correct or not? what is xxx? how to do xxx in KBase?
                text is:
                KBase Documentation

Here you will find guidance for getting started with KBase, full documentation for users and App developers, and help with troubleshooting.

How to cite KBase:

Arkin AP, Cottingham RW, Henry CS, Harris NL, Stevens RL, Maslov S, et a

In [229]:
def generateQA(instruction_prompt, text):
    system_message = SystemMessage(content=prompt.format(input=instruction_prompt))
    human_message = HumanMessage(content=text)
    res = llm([system_message, human_message])
    return res

In [230]:
answers = generateQA(instruction_prompt, documents_html[0].page_content)

In [193]:
answers

AIMessage(content='Generated content: \n    ### question: What is the purpose of the KBase Documentation? ## answer: The KBase Documentation provides guidance for getting started with KBase, full documentation for users and App developers, and help with troubleshooting.\n    ### question: How should KBase be cited in academic work? ## answer: KBase should be cited as follows: Arkin AP, Cottingham RW, Henry CS, Harris NL, Stevens RL, Maslov S, et al. KBase: The United States Department of Energy Systems Biology Knowledgebase. Nature Biotechnology. 2018;36: 566. doi: 10.1038/nbt.4163.\n    ### question: Who funds the work of KBase? ## answer: KBase is funded by the U.S. Department of Energy, Office of Science, Office of Biological and Environmental Research.\n    ### question: What are the award numbers associated with the funding of KBase? ## answer: The award numbers associated with the funding of KBase are DE-AC02-05CH11231, DE-AC02-06CH11357, DE-AC05-00OR22725, and DE-AC02-98CH10886.

In [231]:
def extractQA(answers):
    answers = answers.content.split("###")
    res = []
    for i in range(1, len(answers)):
        q = answers[i].split("##")[0][11:].strip()
        a = answers[i].split("##")[1][9:].strip()
        res.append({'question': q, "answer": a})
    return res
# extractQA(answers)

In [232]:
data = []
for doc in documents_html:
    answers = generateQA(instruction_prompt, doc.page_content)
    try:
        data.extend(extractQA(answers))
    except IndexError:
        print(answers)
        continue
data

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU on tokens_usage_based per min: Limit 10000, Used 8275, Requested 2150. Please try again in 2.55s. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU on tokens_usage_based per min: Limit 10000, Used 7945, Requested 2137. Please try again in 492ms. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU o

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU on tokens_usage_based per min: Limit 10000, Used 7932, Requested 2174. Please try again in 636ms. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU on tokens_usage_based per min: Limit 10000, Used 7987, Requested 2119. Please try again in 636ms. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU o

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU on tokens_usage_based per min: Limit 10000, Used 8252, Requested 2187. Please try again in 2.634s. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU on tokens_usage_based per min: Limit 10000, Used 8114, Requested 1994. Please try again in 648ms. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU 

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU on tokens_usage_based per min: Limit 10000, Used 8140, Requested 2148. Please try again in 1.728s. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU on tokens_usage_based per min: Limit 10000, Used 8212, Requested 2098. Please try again in 1.86s. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU 

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU on tokens_usage_based per min: Limit 10000, Used 8280, Requested 2174. Please try again in 2.724s. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU on tokens_usage_based per min: Limit 10000, Used 8062, Requested 2075. Please try again in 822ms. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU 

...

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU on tokens_usage_based per min: Limit 10000, Used 8145, Requested 2179. Please try again in 1.944s. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU on tokens_usage_based per min: Limit 10000, Used 8564, Requested 2163. Please try again in 4.362s. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU on tokens_usage_based per min: Limit 10000, Used 8100, Requested 2162. Please try again in 1.572s. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU on tokens_usage_based per min: Limit 10000, Used 8250, Requested 2098. Please try again in 2.088s. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU on tokens_usage_based per min: Limit 10000, Used 8113, Requested 2180. Please try again in 1.758s. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU on tokens_usage_based per min: Limit 10000, Used 8092, Requested 2085. Please try again in 1.062s. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU on tokens_usage_based per min: Limit 10000, Used 7872, Requested 2183. Please try again in 330ms. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU on tokens_usage_based per min: Limit 10000, Used 8012, Requested 2191. Please try again in 1.218s. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4 in organization org-zw6y0pzavFVsBHYYRW7WQpeU 

[{'question': 'What can you find in the KBase Documentation?',
  'answer': 'You can find guidance for getting started with KBase, full documentation for users and App developers, and help with troubleshooting.'},
 {'question': 'How to cite KBase in a research paper?',
  'answer': 'You can cite KBase as "Arkin AP, Cottingham RW, Henry CS, Harris NL, Stevens RL, Maslov S, et al. KBase: The United States Department of Energy Systems Biology Knowledgebase. Nature Biotechnology. 2018;36: 566. doi: 10.1038/nbt.4163".'},
 {'question': 'Who funds the work of KBase?',
  'answer': 'The work of KBase is funded by the U.S. Department of Energy, Office of Science, Office of Biological and Environmental Research.'},
 {'question': 'What are the award numbers for the funding received by KBase?',
  'answer': 'The award numbers are DE-AC02-05CH11231, DE-AC02-06CH11357, DE-AC05-00OR22725, and DE-AC02-98CH10886.'},
 {'question': 'What can I find in the "Getting Started" section of KBase?',
  'answer': 'In

In [233]:

output_filepath = 'kbase_formatted_new_more_prompt.json'
with open(output_filepath, 'w') as output_file:
    json.dump(data, output_file, indent=4)
    print(f"Results have been saved to {output_filepath}.")

Results have been saved to kbase_formatted_new_more_prompt.json.
